In [1]:
# System libraries
import sys, os, gc
import datetime

# Math libraries
import math, random
import pandas as pd, numpy as np
import scipy
from scipy import stats
from datetime import timedelta
from datetime import datetime
import itertools

# Data storage libraries
import pickle, sqlite3, simpledbf, boto3

# Custom financial data libraries
import findata_utils as fd
import ml_utils

# Plotting libraries
import matplotlib.pyplot as plt
from matplotlib import rcParams

import warnings
#if not sys.warnoptions:
#    warnings.simplefilter("ignore")

from importlib import reload
fd = reload(fd)

import sklearn as sk
import tensorflow as tf
import xgboost as xgb
import keras

from imblearn.over_sampling import RandomOverSampler

from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.metrics import explained_variance_score, mean_squared_error, confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib 

from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor

from yellowbrick.regressor import ResidualsPlot, PredictionError

# Connect to databases
db = 'C:\\Datasets\\thesis.db'
overleaf = ['C:','Users','bryce','OneDrive','Documents','Overleaf','5babbfe264c952737a9a5117','esc499']
conn = sqlite3.connect(db)
c = conn.cursor()

hdf_path = 'C:\\Datasets\\thesis.h5'
hdf = pd.HDFStore(hdf_path)

   ticker        date action
0  SMU.UN  2019-03-18    Add
1     CJT  2019-03-18    Add
2  DIR.UN  2019-03-18    Add
3     AFN  2019-03-18    Add
4  NWH.UN  2019-03-18    Add

  ticker
0    ABX
1     AC
2    ACB
3  ACO.X
4     AD

  ticker        date       mkt_cap
0    AAV  2010-01-04  1.318031e+09
1    AAV  2010-01-05  1.373801e+09
2    AAV  2010-01-06  1.399827e+09
3    AAV  2010-01-07  1.366365e+09
4    AAV  2010-01-08  1.401686e+09


In [462]:
# Index composition
index = changes.drop_duplicates().set_index(['date','ticker']).unstack()
index = index.fillna(method='ffill').fillna('Delete').unstack().reset_index()
index.drop(['level_0'], axis=1, inplace=True)
index.columns = ['ticker','date','state']
index = index.replace('Delete', 0).replace('Add',1)
index = index.set_index(['date','ticker']).unstack()

# Market capitalizations
mkt_cap = all_mkt_cap.drop_duplicates().pivot_table(index=['date'],columns=['ticker'],values=['mkt_cap'])
contribs = mkt_cap.merge(index, left_on=mkt_cap.index, right_on=index.index, how='left')
contribs['state'] = contribs['state'].fillna(method='ffill')

# Fill the blanks
index=contribs['state']
mkt_cap=contribs['mkt_cap']
index.set_index(contribs['key_0'], inplace=True)
mkt_cap.set_index(contribs['key_0'], inplace=True)

# Find overall market capitalization of the index
contribs = index.multiply(mkt_cap)
contribs = contribs.loc[contribs.sum(axis=1)>0]

mkt_cap = contribs.sum(axis=1)
mkt_cap.name=study_index
mkt_cap.index.name='date'
mkt_cap = pd.DataFrame(mkt_cap).reset_index()
mkt_cap.columns = ['date','SPTSXComp']

mkt_cap.to_sql('index_mkt_cap', conn, if_exists='replace', index=False)

In [463]:
mkt_cap

,date,SPTSXComp
0,2010-01-29,7.001741e+11
1,2010-02-01,7.112851e+11
2,2010-02-02,7.199159e+11
3,2010-02-03,7.229921e+11
4,2010-02-04,7.065905e+11
5,2010-02-05,7.107599e+11
6,2010-02-08,7.028035e+11
7,2010-02-09,7.099281e+11
8,2010-02-10,7.134340e+11
9,2010-02-11,7.217584e+11
